In [4]:
import pandas as pd

# Sample data (replace with your actual data)
data = {
    'Model': ['Golden', 'BART', 'MV-BART', 'BART(DKE)', 'BART(DRD)', 'BART(DTS)', 'BART(DALL)',
              'Golden', 'PGN', 'HMNet', 'PGN(DKE)', 'PGN(DRD)', 'PGN(DTS)', 'PGN(DALL)'],
    'SamSum_Info': [4.37, 3.66, 3.85, 3.88, 3.74, 3.95, 4.05,
                    None, None, None, None, None, None, None],  # Original study, SamSum info. Replace None with your values
    'AMI_Info': [None, None, None, None, None, None, None,
                 4.70, 2.92, 3.52, 3.20, 3.15, 3.05, 3.33],  # Original study, AMI info. Replace None with your values
    'Info*': [None, None, None, None, None, None, None,
               2.4, 2.18, 2.2, 2.18, 3.0, 2.27, 2.52]  # Reproduction experiment. Replace None with your values
}

# Create a DataFrame
df = pd.DataFrame(data)

# Select relevant columns
df_correlation = df[['SamSum_Info', 'AMI_Info', 'Info*']]

# Calculate correlation matrix
correlation_matrix = df_correlation.corr()

# Display correlation matrix
print("Correlation Matrix:")
print(correlation_matrix)


Correlation Matrix:
             SamSum_Info  AMI_Info     Info*
SamSum_Info          1.0       NaN       NaN
AMI_Info             NaN  1.000000  0.012352
Info*                NaN  0.012352  1.000000


In [5]:
import pandas as pd

# Sample data (replace with your actual data)
data = {
    'Model': ['Golden', 'BART', 'MV-BART', 'BART(DKE)', 'BART(DRD)', 'BART(DTS)', 'BART(DALL)',
              'Golden', 'PGN', 'HMNet', 'PGN(DKE)', 'PGN(DRD)', 'PGN(DTS)', 'PGN(DALL)'],
    'SamSum_Info': [4.37, 3.66, 3.85, 3.88, 3.74, 3.95, 4.05,
                    None, None, None, None, None, None, None],  # Replace None with your values
    'AMI_Info': [None, None, None, None, None, None, None,
                 4.70, 2.92, 3.52, 3.20, 3.15, 3.05, 3.33],  # Replace None with your values
    'Info*': [None, None, None, None, None, None, None,
               2.4, 2.18, 2.2, 2.18, 3.0, 2.27, 2.52]  # Replace None with your values
}

# Create a DataFrame
df = pd.DataFrame(data)

# Select relevant columns for correlation
df_correlation = df[['AMI_Info', 'Info*']]

# Drop rows with NaN values
df_correlation = df_correlation.dropna()

# Calculate correlation coefficient
correlation_coefficient = df_correlation['AMI_Info'].corr(df_correlation['Info*'])

print("Pearson Correlation Coefficient between AMI_Info and Info*:", correlation_coefficient)


Pearson Correlation Coefficient between AMI_Info and Info*: 0.012351910068674858


In [6]:
df_correlation

,AMI_Info,Info*
7,4.70,2.40
8,2.92,2.18
9,3.52,2.20
10,3.20,2.18
11,3.15,3.00
12,3.05,2.27
13,3.33,2.52


The coefficient value of -0.058 indicates a weak negative correlation between the variables "AMI_Info" and "Info*".
A negative correlation means that as the value of one variable increases, the value of the other variable tends to decrease.
The magnitude of the correlation coefficient (in this case, 0.058) suggests that the relationship between the variables is very weak.
In summary, the Spearman correlation coefficient provides insight into how closely related two variables are, and in this case, there is a weak negative relationship between "AMI_Info" and "Info*"

In [7]:
correlation_coefficient_spearman = df_correlation['AMI_Info'].corr(df_correlation['Info*'],method='spearman')
print("Spearman Correlation Coefficient between AMI_Info and Info*:", correlation_coefficient_spearman)

Spearman Correlation Coefficient between AMI_Info and Info*: 0.2702812388086677


In [8]:


# -*- coding: utf-8 -*-
"""This code computes the coefficient of variation (CV) and some other stats for small samples (indicated by the * added to CV)
for a given set of measurements which are assumed to be for the same or similar object, using the same measurand.
Stats are adjusted for small sample size. Paper ref: Belz, Popovic & Mille (2022) Quantified Reproducibility Assessment of NLP Results,
ACL'22.

In this self-contained version, the set of measurements on which CV is computed is assigned to the variable set_of_set_of_measurements
(see examples in code below).

The reproducibility stats reported in the output are:
* the unbiased coefficient of variation
* the sample mean
* the unbiased sample standard deviation with 95% confidence intervals, estimated on the basis of the standard error of the unbiassed sample variance
* the sample size
* the percentage of measured valued within two standard deviations
* the percentage of measured valued within one standard deviation

Example narrative output:

The unbiased coefficient of variation is 1.5616560359100269 \
for a mean of 85.58285714285714 , \
unbiased sample standard deviation of 1.2904233075765223 with 95\% CI (0.4514829817654973, 2.1293636333875474) ,\
and a sample size of 7 . \
100.0 % of measured values fall within two standard deviations. \
71.429 % of measured values fall within one standard deviation.

NOTE:
* CV assumes all measurements are positive; if they're not, shift measurement scale to start at 0
* for fair comparison across studies, measurements on a scale that doesn't start at 0 need to be shifted to a scale that does start at 0

KNOWN ISSUES:

none
"""

import math
import numpy as np
from scipy.stats import t

# measurements from Belz et al. (2022)
# --- Table 4: BLEU scores for 7 NTS_def repros ---
#set_of_set_of_measurements = [[84.51, 84.50, 87.46, 85.60, 84.20, 86.61, 86.20]] # 7, NTS_def, BLEU
# --- Table 4: SARI scores for 5 NTS_def repros ---
#set_of_set_of_measurements = [[30.65, 30.65, 29.13, 30.65, 29.96]] # 5, NTS_def, SARI
# --- Table 4: BLEU scores for 6 NTS-w2v_def repros ---
#set_of_set_of_measurements = [[87.50, 80.75, 89.36, 88.10, 89.64, 88.80]] # 6, NTS-w2v_def, BLEU
# --- Table 4: SARI scores for 4 NTS_w2v_def repros ---
#set_of_set_of_measurements = [[31.11, 30.28, 31.11, 29.12]] # 4, NTS-w2v_def, SARI
# --- Table 5: blue highlights (i.e. subsets of BLEU scores from above where outputs were reused, not regenerated) ---
#set_of_set_of_measurements = [[84.51, 84.50, 85.60, 84.20]] # 7, NTS_def, BLEU
#set_of_set_of_measurements = [[87.50, 89.36, 88.10]] # 6, NTS-w2v_def, BLEU
# --- Section 4.1, fourth paragraph: subset of scores where outputs were regenerated and Nisioi's BLEU script was used
#set_of_set_of_measurements = [[84.51, 87.46, 86.61], # 7, NTS_def, BLEU
#                              [30.65, 29.13, 29.96], # 5, NTS_def, SARI
#                              [87.50, 80.75, 89.64], # 6, NTS-w2v_def, BLEU
#                              [31.11, 30.28, 29.12]] # 4, NTS-w2v_def, SARI
# --- Table 6: wF1 scores for 11 system variants ---
#set_of_set_of_measurements = [[0.428, 0.493, 0.426, 0.574, 0.579, 0.590, 0.574, 0.600],
#                              [0.721, 0.603, 0.605, 0.606, 0.720, 0.732, 0.606, 0.740],
#                              [0.719, 0.604, 0.607, 0.607, 0.723, 0.733, 0.607, 0.736],
#                              [0.726, 0.681, 0.680, 0.680, 0.722, 0.728, 0.680, 0.732],
#                              [0.724, 0.680, 0.680, 0.681, 0.725, 0.729, 0.681, 0.731],
#                              [0.703, 0.660, 0.650, 0.651, 0.699, 0.711, 0.651, 0.710],
#                              [0.693, 0.661, 0.652, 0.653, 0.699, 0.712, 0.653, 0.716],
#                              [0.449, 0.600, 0.433, 0.597, 0.635, 0.646, 0.597, 0.698],
#                              [0.471, 0.647, 0.447, 0.647, 0.696, 0.711, 0.647, 0.726],
#                              [0.693, 0.658, 0.683, 0.668, 0.692, 0.689, 0.659, 0.391],
#                              [0.689, 0.662, 0.681, 0.659, 0.681, 0.684, 0.657, 0.401]]

#set_of_set_of_measurements = [[2.4, 2.175, 2.2, 2.175, 3.0, 2.27, 2.52]] #AMI score

set_of_set_of_measurements = [
  [4.7, 2.4], # Gold
  [2.92, 2.18], # PGN
  [3.52, 2.2], # HMNet
  [3.20, 2.18], # PGN(DKE)
  [3.15, 3.0], # PGN(DRD)
  [3.05, 2.27], # PGN(DTS)
  [3.33, 2.52], #PGN(DALL)
]


for set_of_measurements in set_of_set_of_measurements:
  if len(set_of_measurements) < 2:
    print(set_of_measurements, ": set of measurements is smaller than 2")
    break

  sample_mean = np.mean(set_of_measurements)
  if sample_mean <= 0:
    print(set_of_measurements, ": mean is 0 or negative")
    break

  sample_size = len(set_of_measurements)
  degrees_of_freedom = sample_size-1
  sum_of_squared_differences = np.sum(np.square(sample_mean-set_of_measurements))

  # unbiassed sample variance s^2
  unbiassed_sample_variance = sum_of_squared_differences/degrees_of_freedom
  # corrected sample standard deviation s
  corrected_sample_standard_deviation = np.sqrt(unbiassed_sample_variance)
  # Gamma(N/2)
  gamma_N_over_2 = math.gamma(sample_size/2)
  # Gamma((N-1)/2)
  gamma_df_over_2 = math.gamma(degrees_of_freedom/2)
  # c_4(N)
  c_4_N = math.sqrt(2/degrees_of_freedom)*gamma_N_over_2/gamma_df_over_2
  # unbiassed sample std dev s/c_4
  unbiassed_sample_std_dev_s_c_4 = corrected_sample_standard_deviation/c_4_N
  # standard error of the unbiassed sample variance (assumes normally distributed population)
  standard_error_of_unbiassed_sample_variance = unbiassed_sample_variance*np.sqrt(2/degrees_of_freedom)
  # estimated std err of std dev based on std err of unbiassed sample variance
  est_SE_of_SD_based_on_SE_of_unbiassed_sample_variance = standard_error_of_unbiassed_sample_variance/(2*unbiassed_sample_std_dev_s_c_4)

  # COEFFICIENT OF VARIATION CV
  coefficient_of_variation = (unbiassed_sample_std_dev_s_c_4/sample_mean)*100
  # SMALL SAMPLE CORRECTED COEFFICIENT OF VARIATION CV*
  small_sample_coefficient_of_variation = (1+(1/(4*sample_size)))*coefficient_of_variation

  # compute percentage of measured values within 1 and 2 standard deviations from the mean
  # initialise counts
  count_within_1_sd = 0
  count_within_2_sd = 0
  # for each measured value
  for m in set_of_measurements:
    # if it's within two std devs, increment count_within_2_sd
    if np.abs(m-sample_mean) < 2*unbiassed_sample_std_dev_s_c_4:
      count_within_2_sd += 1
      #if it's also within one std devs, increment count_within_1_sd
      if np.abs(m-sample_mean) < unbiassed_sample_std_dev_s_c_4:
        count_within_1_sd += 1

  # report results as described in code description above
  print("The unbiased coefficient of variation is",small_sample_coefficient_of_variation)
  print("for a mean of",sample_mean,", ")
  print("unbiased sample standard deviation of",unbiassed_sample_std_dev_s_c_4,", with 95\% CI",t.interval(0.95, degrees_of_freedom, loc=unbiassed_sample_std_dev_s_c_4, scale=est_SE_of_SD_based_on_SE_of_unbiassed_sample_variance),",")
  print("and a sample size of",sample_size,".")
  print(count_within_2_sd/sample_size*100,"% of measured values fall within two standard deviations.")
  print(round(count_within_1_sd/sample_size*100, 3),"% of measured values fall within one standard deviation.", )




The unbiased coefficient of variation is 64.59470900307073
for a mean of 3.55 , 
unbiased sample standard deviation of 2.038321928541343 , with 95\% CI (-9.620475344897981, 13.697119201980668) ,
and a sample size of 2 .
100.0 % of measured values fall within two standard deviations.
100.0 % of measured values fall within one standard deviation.
The unbiased coefficient of variation is 28.93270256625179
for a mean of 2.55 , 
unbiased sample standard deviation of 0.6558079248350406 , with 95\% CI (-3.095283371836741, 4.406899221506822) ,
and a sample size of 2 .
100.0 % of measured values fall within two standard deviations.
100.0 % of measured values fall within one standard deviation.
The unbiased coefficient of variation is 46.01562882158549
for a mean of 2.8600000000000003 , 
unbiased sample standard deviation of 1.1698195415976402 , with 95\% CI (-5.5213162848979715, 7.860955368093252) ,
and a sample size of 2 .
100.0 % of measured values fall within two standard deviations.
100.0 %